In [11]:
#B) Anwendung: Machen Sie sich mit der Dokumentation vertraut und schreiben Sie Beispiel-Code für a)
#Indexierung und b) Querying in einem Notebook.


In [12]:
""" Verbindung zu Milvus herstellen:
"localhost" bedeutet, dass Milvus auf dem eigenen Rechner läuft. Port 19530 ist der Standardport, über den die Verbindung erfolgt."""

# 1. Imports und Server-Setup
from milvus import default_server
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

# Server starten
default_server.start()

# Mit dem gestarteten Server verbinden
connections.connect(
    alias="default",
    host='127.0.0.1',
    port=default_server.listen_port
)

print("✓ Server gestartet und Verbindung hergestellt")

KeyboardInterrupt: 

In [10]:
"""# 2. Schema für die Collection definieren
# Ein Schema beschreibt die Struktur der Daten.
# Hier definieren wir Felder (fields) mit Namen und Typen.
# - id: eindeutige Identifikation (ganzzahlig, Primärschlüssel)
# - embedding: Vektor (hier 3D zum Beispiel)
# - englisch: englisches Wort als Text
# - deutsch: deutsches Wort als Text"""
# 2. Schema für die Collection definieren
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=3),  # 3D-Vektor für Einfachheit
    FieldSchema(name="englisch", dtype=DataType.VARCHAR, max_length=50),
    FieldSchema(name="deutsch", dtype=DataType.VARCHAR, max_length=50)
]

"""# Schema zusammenstellen mit Beschreibung"""
schema = CollectionSchema(fields, description="Englisch-Deutsch Vektor Wörterbuch")

"""
# 3. Collection erstellen mit dem definierten Schema"""
collection = Collection("englisch_deutsch_dict", schema)

"""# 4. Beispiel-Daten vorbereiten: IDs, Embeddings, Wörter"""
# Wörter und 3D-Embeddings (hier einfache Zahlen statt echte Vektor-Embeddings)
data = [
    [1, 2, 3],  # IDs
    [
        [0.1, 0.2, 0.3],  # embedding von "cat"
        [0.2, 0.1, 0.0],  # embedding von "dog"
        [0.4, 0.4, 0.5]   # embedding von "papaya"
    ],
    ["cat", "dog", "papaya"],      # englische Wörter
    ["Katze", "Hund", "Papaya"]    # deutsche Übersetzung
]

"""5. Daten in die Collection einfügen"""
collection.insert(data)

"""# 6. Index auf Embeddings erstellen für schnelle Suche (z.B. IVF_FLAT, L2 Abstand)"""
index_params = {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 2}}
collection.create_index("embedding", index_params)


"""# 7. Collection laden in den Speicher ( damit Suchanfragen funktionieren)"""
collection.load()

""" 8. Suche durchführen: Ähnlichster Vektor zu [0.1, 0.2, 0.25] (ähnlich zu "cat") """
search_vectors = [[0.1, 0.2, 0.25]]

# Such-Parameter: Vektor-Feld "embedding", Metrik "L2" (Abstand), max 1 Ergebnis
results = collection.search(
    data=search_vectors,
    anns_field="embedding",
    param={"metric_type": "L2", "params": {"nprobe": 1}},
    limit=1,
    output_fields=["englisch", "deutsch"]
)

"""9. Ergebnisse anzeigen"""
for hits in results:
    for hit in hits:
        print(f"Englisch: {hit.entity.get('englisch')} - Deutsch: {hit.entity.get('deutsch')} - Distanz: {hit.distance}")

Englisch: cat - Deutsch: Katze - Distanz: 0.002500001108273864
